### to put in readme: 
# Dr. Long's solutions: hosted in data/class/ecoevo283/public as python scripts
General steps to take: have to copy the data as he stated in the .txt- you must make DNAseq, RNAseq, 
and ATACseq directories, and put empty rawdata directories within them.


### This differs from the symlink_script file because I modified the paths for use in my home directory;
which was ~/EE283_files, where I had a symlink to pdgardne created there.
I actually think those paths might work; the key issue was where, exactly, to have the scripts so they would run. Refer to the for() loop for the DNAseq solution to see.

## DNAseq solution below:               

So this is following more closely the procedure we went over in class; where you manually made ATACseq,DNAseq, RNAseq, and made 'rawdata' directories 

In [2]:
#both the paths of input and output are the same-- so I would expect this to be a 
#renaming, and not symlinking to another directory?


#scratch the above- this has nothing to do with that, actually. This script makes, in *whichever* directory you are currently
#in, simlinks to the DNA data there.

#Must run this in the directory above your empty DNAseq directory, (in my case, pdgardne, symlinked to a folder in my hpc3 base) 
#which should contain an empty rawdata directory- mirroring the structure of the data on the 
#public directory


import os
files=os.listdir("/data/class/ecoevo283/public/RAWDATA/DNAseq")
for f in files:
        if f.endswith('.fq.gz'):
                os.symlink("/data/class/ecoevo283/public/RAWDATA/DNAseq/"+f, "DNAseq/rawdata/"+f) 

FileNotFoundError: [Errno 2] No such file or directory: '/data/class/ecoevo283/public/RAWDATA/DNAseq/ADL09_1_2.fq.gz' -> 'DNAseq/rawdata/ADL09_1_2.fq.gz'

## RNAseq solution: 

In [ ]:
#do some stuff from the command line first
#get a list of all the fastq files in the entire path
#taken from Dr. Long's files

# I originally had paths changed below to fit my own directory, which has symlinks to the class one
# You could probably do it that way, but I misunderstood- I had the shell code and python script with
# paths to my own, empty directories, when you need to have these scripts and run them ABOVE the 
# directory where things are happening, pulling the information from wherever the data is STORED,
# not where you want things to go. 


find /data/class/ecoevo283/public/RAWDATA/RNAseq/RNAseq384plex_flowcell01/ -name "*.fastq.gz" >fastqs.txt

#create a table that will allow for sample name lookups..

# (again, must do in command line) 
cat /data/class/ecoevo283/public/RAWDATA/RNAseq/RNAseq384_SampleCoding.txt | cut -f1,12 | tail -n +2 >dict.txt
#this throws an error on one file but seems to work otherwise

# rename.RNAseq.py SHOULD BE the script that contains your python code. 
python rename.RNAseq.py <fastqs.txt #this executes your renaming symlink script by reading in the indices of fastqs.txt


In [ ]:
#taken from Dr. Long's files

#RUN THIS IN the directory above the RNAseq directory. Not sure why it doesn't work
#one level down.

#contents of rename.RNAseq.py.
#create a table that will allow for sample name lookups..



import os
import re
import sys

###read in tab delimited sample# \t samplename 
#make a dictionary, read in the file "dict.txt" and for each line in the file, split the line and call the first item, the key, to create the dictionary 'value', so that if you type d it will tell you the value

d = {}
with open("dict.txt") as FILEIN:
        for line in FILEIN:
                (key, val) = line.split()
                d[int(key)] = val

###make symlinks
for f in sys.stdin: #f will take on the first value for the path for each sample
        f = f.strip('\n') #strip the line return we don't want
        if f.endswith('.fastq.gz') and "Undetermined" not in f:
                num=re.search("[0-9]+_",f).group(0).replace("_","")
                dir=re.search("_R[0-9]_",f).group(0).replace("_R1_","F").replace("_R2_","R")
                out="RNAseq/rawdata/"+ d[int(num)] + "_" + dir +".fq.gz" #this looks up the value of the value from the table
                print(f,num,dir,out) #print everything out to the screen to debug 
                os.symlink(f, out)

 ## ATACseq solution:

In [3]:
####### example sample: Sample_ACCAGCA-TATGCAGT_4R009_L1_P059_R2.fq.gz
#again, taken from Dr. Long's files

#again, run in the directory above the ATACseq/rawdata directories you must have
import os
import re
files=os.listdir("/data/class/ecoevo283/public/RAWDATA/ATACseq")
for f in files:
        if f.endswith('.fq.gz'):
                f2=re.search("P[0-9]+_R[0-9]",f).group(0).replace("R","").replace("_1","_F").replace("_2","_R")
                os.symlink("/data/class/ecoevo283/public/RAWDATA/ATACseq/"+f, "ATACseq/rawdata/" + f2 + ".fq.gz")

FileNotFoundError: [Errno 2] No such file or directory: '/data/class/ecoevo283/public/RAWDATA/ATACseq/Sample_ACCAGCA-TATGCAGT_4R009_L1_P059_R2.fq.gz' -> 'ATACseq/rawdata/P059_R.fq.gz'

### my attempts below that don't work:

In [ ]:
# DNAseq README:  "ADLxx_yy_zz.fq.gz -> xx = DNA source, yy = replicate lane, zz = F or R read"
# Goal: maybe symlink each file to a new directory? Rename? 
#I'm realizing the the .txt we got for this class might be saying 'DNAseq should be fine' as in, 
#it doesn't need any alterations



In [ ]:
#RNAseq is where the work seems to need to happen
#in class we mentioned using dictionaries, regular expressions



"""
RNAseq README  contents:

FILE = RNAseq384_SampleCoding.xlsx

Column names/meanings:

SampleNumber = 1-376 numeric code.

Multiplex_i5index = 48-plex for each sample (plex1-plex8) and the i5 index that was associated with each plex. (Note however that the i5 index was only used for the preliminary 384-plex, dual-index run, so is irrelevant here.)

Lane = Lane each plex was run in (L001-L008). Note that plex1 was always run in lane L001, and so on.

i7index = i7 index associated with each sample within a plex (i7 indices are re-used across plexes).

PlateName = There are 4 plates (BodyPlate, EmbyroPlate, HeadPlate, and PupaPlate). Note that the tissue in the plate name refers to the majority tissue on the plate; NOT all samples on a plate are from the same tissue (see "TissueCode" column).

PlateRow =
PlateColumn =
PlateWell = Sample positions in each 96-well plate.

RILcode = Standard 5-digit RIL code.

"""
#data structure: RNAseq384plex_flowcell1101 contains different project directories, 
#and within each project dir there are sample directories, which contain (what I think are?) 
#forward and reverse reads of each sample in gzipped fastq format. so we'd have to go 3 steps 
#down in directories to actually be able to access the files

#the flowcell1011 directories all lead to samples, even the Undetermined_indices one

#.txt mentions shutil.copy or os.symlink()

#os.symlink(source,destination) #https://www.tutorialspoint.com/python3/os_symlink.htm
    #OR 
    #from shutil import copyfile
    #copyfile(source,destination)


"""
from shutil import copyfile

for directory in RNAseq384plex_flowcell101/*:
    copyfile(RNAseq384plex_flowcell101/Project*/Sample*, <directory we copy to>)""" # not sure how to specify all directories in a directory... maybe this? https://unix.stackexchange.com/questions/86722/how-do-i-loop-through-only-directories-in-bash
    for /Project*
        for /Sample*
        copyfile(*.fastq.gz, <directory we copy to>) #this would just dump all the samples together..
"""      

In [1]:


#As stated in the .txt file for fastqc number for a ATACseq file 

bash script:
"""
module avail fastqc
module load fastqc/0.11.9
fastqc --help
"""

#we want to make a copy of the files so we can gunzip them and then 
#we would want to then do:
from shutil import copyfile
for file in ~/EE283_files/pdgardne/rawdata/ATACseq/*.fq.gz: 
    gunzip *.fq.gz
    fastqc -f fastq ${file} #https://wiki.bits.vib.be/index.php/Linux_command_line

SyntaxError: invalid syntax (<ipython-input-1-a2d080c11b27>, line 3)

Dr. Long says solutions should be in public repository. He suggests running python scripts from the command line without using the interactive python 

